# 🎙️ Sauti Ya Kenya - TTS Training

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Msingi-AI/Sauti-Ya-Kenya/blob/main/notebooks/sauti_ya_kenya_training.ipynb)

This notebook trains the Kenyan Swahili TTS model with optimized memory usage and GPU acceleration.

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create project directories
!mkdir -p /content/drive/MyDrive/Sauti-Ya-Kenya/{data,checkpoints,logs}

In [ ]:
# Clean up and clone repository
%cd /content
!rm -rf Sauti-Ya-Kenya
!git clone https://github.com/Msingi-AI/Sauti-Ya-Kenya.git
%cd Sauti-Ya-Kenya

# Install dependencies
!pip install -r requirements.txt

# Set up symlinks to Drive
!rm -rf data checkpoints
!ln -s /content/drive/MyDrive/Sauti-Ya-Kenya/data data
!ln -s /content/drive/MyDrive/Sauti-Ya-Kenya/checkpoints checkpoints

# Verify setup
!ls -la

In [ ]:
# Configure PyTorch memory settings
import os
import torch

# Set memory optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:32'

# Clear CUDA cache
torch.cuda.empty_cache()

# Print GPU info
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
    print(f"Memory Allocated: {torch.cuda.memory_allocated() / 1024**3:.1f}GB")
    print(f"Memory Cached: {torch.cuda.memory_reserved() / 1024**3:.1f}GB")
else:
    print("No GPU available!")

In [ ]:
# Check and setup data
import os

drive_data = '/content/drive/MyDrive/Sauti-Ya-Kenya/data'
if not os.path.exists(drive_data) or not os.listdir(drive_data):
    print("No data found. Please upload data.zip when prompted.")
    from google.colab import files
    uploaded = files.upload()
    
    if 'data.zip' in uploaded:
        !unzip data.zip -d {drive_data}
        print("\nData files:")
        !ls -R {drive_data}
    else:
        raise ValueError("Please upload data.zip")
else:
    print("Found existing data:")
    !ls -R {drive_data}

In [ ]:
# Preprocess data if needed
if not os.path.exists('processed_data'):
    print("Preprocessing data...")
    !python -m src.preprocess_data
    print("\nProcessed files:")
    !ls -R processed_data
else:
    print("Using existing preprocessed data")

In [ ]:
# Find latest checkpoint
import glob
checkpoints = glob.glob("/content/drive/MyDrive/Sauti-Ya-Kenya/checkpoints/*.pt")
resume_path = None

if checkpoints:
    resume_path = max(checkpoints, key=os.path.getctime)
    print(f"Found checkpoint: {resume_path}")
    print("\nAll checkpoints:")
    for cp in sorted(checkpoints):
        size_mb = os.path.getsize(cp) / (1024 * 1024)
        print(f"- {os.path.basename(cp)} ({size_mb:.1f}MB)")
else:
    print("No existing checkpoints. Starting fresh training.")

In [ ]:
# Start training with memory optimization
cmd = "python -m src.train --batch_size 8 --grad_accum 4"
if resume_path:
    cmd += f" --resume {resume_path}"

print(f"Running: {cmd}")
!{cmd}